# Simple Bot

To run a bot locally we need three things:
1. a mechanism to redirect a public URL used for a webhook to the local machine
2. a local webserver serving the POSTs to the redirected webhook
3. some bot logic which parses the message posted by the user and acts on it

## Automating the use of ngrok to tunnel webhooks to local machine


Starting `ngrok` using `ngrok http 80 -log=stdout -log-format=json -log-level=info` creates the following JSON output on stdout:

```
{"lvl":"info","msg":"no configuration paths supplied","t":"2017-06-.."}
{"err":"stat /Users/jkrohn/.ngrok2/ngrok.yml: no such file or directory","lvl":"info","msg":"ignoring default config path, could not stat it","path":"/Users/jkrohn/.ngrok2/ngrok.yml","t":"2017-06-.."}
{"comp":"memory storage","lvl":"info","msg":"run component","obj":"controller","t":"2017-06-.."}
{"comp":"signal handler","lvl":"info","msg":"run component","obj":"controller","t":"2017-06-.."}
{"comp":"Tunnel session","lvl":"info","msg":"run component","obj":"controller","t":"2017-06-.."}
{"comp":"updater","lvl":"info","msg":"run component","obj":"controller","t":"2017-06-.."}
{"comp":"web","lvl":"info","msg":"run component","obj":"controller","t":"2017-06-.."}
{"addr":"127.0.0.1:4040","lvl":"warn","msg":"can't bind default web address, trying alternatives","obj":"web","t":"2017-06-.."}
{"addr":"127.0.0.1:4041","lvl":"info","msg":"starting web service","obj":"web","t":"2017-06-.."}
{"lvl":"info","msg":"tunnel session started","obj":"tunSess","t":"2017-06-.."}
{"id":"44a534c2b867","lvl":"info","msg":"client session established","obj":"csess","t":"2017-06-.."}
```

The idea to automate the use of `ngtok` for Python is to start an `ngrok` instance from Python as a subprocess and then parse the above startup messages to:
* learn the port of the `ngrok` admin interface (in case multiple `ngrok` instances are running on the same host). This port is included in the `addr` part of this message:
`{"addr":"127.0.0.1:4041","lvl":"info","msg":"starting web service","obj":"web","t":"2017-06-.."}`
* wait until the client session with the `ngrok` service has been established. This is indicated by this message:
`{"id":"44a534c2b867","lvl":"info","msg":"client session established","obj":"csess","t":"2017-06-.."}`

The output of the  `ngrok` process needs to be read continuously to avoid that the process locks up. This needs to be  done in a separate thread.

In [1]:
import threading
import shutil
import subprocess
import requests
import time
import json

class Ngrok(threading.Thread):
    ''' Ngrok: class to automate starting a local ngrok instance as a subprocess.
    '''
    
    def __init__(self, port=None):
        '''Initalize Ngrok tunnel.

        :param port: int, localhost port forwarded through tunnel

        '''
        assert shutil.which("ngrok"), "ngrok command must be installed, see https://ngrok.com/"
        threading.Thread.__init__(self)
        
        self.port = port
        return
    
    def read_json_from_ngrok(self):
        ''' read stdout of ngrok process and try to parse as JSON
        
            returns: decoded JSON of single ngrok output line
        '''
        while True:
            line = self.ngrok.stdout.readline().decode()
            # try to parse as JSON
            try:
                line = json.loads(line)
            except json.JSONDecodeError:
                # ignore anything that isn't JSON
                continue
            break
        return line
        
    def start(self):
        ''' 
        start a local ngrok process
        start a thread in the background to read stdout of the ngrok process
        As soon as the nrgok client connection has been established determine the public URL and report that back
        '''
        
        # where is ngrok?
        ngrok = shutil.which('ngrok')
        
        # commandline to start ngrok
        cmd = '{} http {} -log=stdout -log-format=json -log-level=info'.format(ngrok, self.port)
        
        # start ngrok process
        self.ngrok = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)

        # give it some time
        time.sleep(.5)
        assert self.ngrok.poll() is None, "ngrok failed to start"
        
        ngrok_addr = None
        print('Waiting for ngrok startup...')
        while True:
            line = self.read_json_from_ngrok()
            
            # {"addr":"127.0.0.1:4041","lvl":"info","msg":"starting web service","obj":"web","t":"2017-06-.."}
            if (line.get('obj') == 'web' and 
                line.get('lvl') == 'info' and 
                line.get('msg') == 'starting web service'):
                # get ngrok admin interface address from message
                ngrok_addr = line['addr']
            # {"id":"44a534c2b867","lvl":"info","msg":"client session established","obj":"csess","t":"2017-06-.."}
            if (line.get('obj') == 'csess' and 
                line.get('lvl') == 'info'):
                # startup done: terminate loop
                break

        # now as the ngrok client is up we can try to use the ngrok client API to get the public address of the tunnel
        # might take some time for the tunnels to come up; hence we try repeatedly until the API call succeeds
        
        ''' the expected JSON result looks something like this:
            {
                "tunnels": [
                    {
                        "proto": "https",
                        "name": "command_line",
                        "config": {
                            "addr": "localhost:60176",
                            "inspect": true
                        },
                        "metrics": {
                            ...
                        },
                        "public_url": "https://47bff724.ngrok.io",
                        "uri": "/api/tunnels/command_line"
                    },
                    {
                        "proto": "http",
                        "name": "command_line (http)",
                        "config": {
                            "addr": "localhost:60176",
                            "inspect": true
                        },
                        "metrics": {
                            ...
                        },
                        "public_url": "http://47bff724.ngrok.io",
                        "uri": "/api/tunnels/command_line+%28http%29"
                    }
                ],
                "uri": "/api/tunnels"
            }
        
        '''
        while True:
            print('Trying to get tunnel information from ngrok client API')
            response = requests.get('http://{}/api/tunnels'.format(ngrok_addr), headers={'content-type':'application/json'}).json()
            if response.get('tunnels'):
                break
            time.sleep(0.5)
        
        # Default: take the 1st URL    
        url = response['tunnels'][0]['public_url']
        
        # but we prefer HTTPS if an HTTPS tunnel exists
        https_tunnel = next((t for t in response['tunnels'] if t['proto'] == 'https'), None)
        if https_tunnel is not None:
            url = https_tunnel['public_url']
            
        # Now start the actual Thread
        threading.Thread.start(self) 
        
        # return the public URL
        return url

    def stop(self):
        """Tell ngrok to tear down the tunnel.

        Stop the background tunneling process.
        """
        self.ngrok.terminate()
        return
        
    def run(self):
        # continuously read from the ngrok process output to prevent the process from blocking
        while True:
            self.read_json_from_ngrok()
        return       

## Putting the pieces together

First a bot needs to be registered at the [Webex Developer Portal](https://developer.webex.com/). As part of the bot registration an `access token` is created. This access token needs to be used by the Bot code for all authenticated requests to the Webex APIs. The bot `access token` has a virtually unlimited lifetime and does not need to be refreshed.

For the bot logic (webserver and act on messages) we are using the readily available Python module [webexteamsbot](https://github.com/hpreston/webexteamsbot) 

In [2]:
import webexteamsbot
import webexteamssdk
import os
import functools
import werkzeug.serving

# Local port for the Websocket
LOCAL_PORT = 5000

# insert your Bot access token here. This token is created at developer.cisco.com under 'My Apps'
BOT_ACCESS_TOKEN = os.environ.get('BOT_ACCESS_TOKEN')
if BOT_ACCESS_TOKEN is None:
    BOT_ACCESS_TOKEN = '<insert your token here>'
assert BOT_ACCESS_TOKEN

BOT_EMAIL = os.environ.get('BOT_EMAIL')
if BOT_EMAIL is None:
    BOT_EMAIL = '<insert your bot email here>'
assert BOT_EMAIL

BOT_APP_NAME = os.environ.get('BOT_APP_NAME')
if BOT_APP_NAME is None:
    BOT_APP_NAME = '<insert your app name here>'


ngrok = Ngrok(port=LOCAL_PORT)
ngrok_url = ngrok.start()
print('Public ngrok URL: {}'.format(ngrok_url))

# Create a new bot
bot = webexteamsbot.TeamsBot(BOT_APP_NAME, teams_bot_token=BOT_ACCESS_TOKEN,
                             teams_bot_url=ngrok_url, teams_bot_email=BOT_EMAIL, debug=True)

# Run Bot
werkzeug.serving.run_simple(hostname='0.0.0.0', port=LOCAL_PORT, application=bot)


Waiting for ngrok startup...
Trying to get tunnel information from ngrok client API
Trying to get tunnel information from ngrok client API
Public ngrok URL: https://a4b81eb9.ngrok.io


Teams Bot Email: brkcol2175@webex.bot
Teams Token: REDACTED
Found existing webhook.  Updating it.
Configuring Webhook. 
Webhook ID: Y2lzY29zcGFyazovL3VzL1dFQkhPT0svZTc4OGVmN2MtYjI3ZS00YjQwLWE0MWItZWRlZjRmZGU5MjNj
 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
Message content:
Webex Teams Message:
{
  "id": "Y2lzY29zcGFyazovL3VzL01FU1NBR0UvZWU3ZDc4ZTAtNDE5ZS0xMWVhLTkwMzgtZWIxZjFjZTk4ODRl",
  "roomId": "Y2lzY29zcGFyazovL3VzL1JPT00vNjkyYzIzODAtNDE5ZS0xMWVhLTk0ZGEtNGIzMjE0YzIyMWVh",
  "roomType": "group",
  "text": "Testbot, hi",
  "personId": "Y2lzY29zcGFyazovL3VzL1BFT1BMRS8wNzhkOGVjMi05Mjg5LTQ2NTUtOWE5NC0wNDNiOWVjMTMyOTk",
  "personEmail": "jkrohn@cisco.com",
  "html": "<p><spark-mention data-object-type=\"person\" data-object-id=\"Y2lzY29zcGFyazovL3VzL1BFT1BMRS9iNzczYWFmMC1lZDBiLTQ0MjctODk3ZC1mNmVlN2VmNmE0ZWE\">Testbot</spark-mention>, hi</p>",
  "mentionedPeople": [
    "Y2lzY29zcGFyazovL3VzL1BFT1BMRS9iNzczYWFmMC1lZDBiLTQ0MjctODk3ZC1mNmVlN2VmNmE0ZWE"
  ],
  "created": "2020-

## Adding some more stuff/fun

There are lots of APIs available which can be used as a source of information. Using Python we can use APIs directly, scrape content from web pages or use available Python modules to gather information. Using the `ciscosparkbot` module we can simply add commands using the `add_command()` method. For each command we need to define a command string, a help text, and a callback function to be called if the command is entered.

In [4]:
import webexteamsbot
import webexteamssdk
import os
import functools
import werkzeug.serving
import logging
import re
import requests
import requests_toolbelt
import urllib.parse
import random

from bs4 import BeautifulSoup
import random


# Local port for the Websocket
LOCAL_PORT = 5000

# insert your Bot access token here. This token is created at developer.cisco.com under 'My Apps'
BOT_ACCESS_TOKEN = os.environ.get('BOT_ACCESS_TOKEN')
if BOT_ACCESS_TOKEN is None:
    BOT_ACCESS_TOKEN = '<insert your token here>'
assert BOT_ACCESS_TOKEN

BOT_EMAIL = os.environ.get('BOT_EMAIL')
if BOT_EMAIL is None:
    BOT_EMAIL = '<insert your bot email here>'
assert BOT_EMAIL
    
BOT_APP_NAME = os.environ.get('BOT_NAME')
if BOT_APP_NAME is None:
    BOT_APP_NAME = '<insert your app name here>'
assert BOT_APP_NAME
    
    
def get_joke(message):
    """
    Act on the /chuck command. Get a random Chuck Norris joke from the Internet Chuck Norris Database
    API documentation: http://www.icndb.com/api/
    :param api: Spark API instance
    :param message: message object
    :return: markdown of text to be posted
    """
    # get a random Chuck Norris joke
    # r = requests.get('http://api.icndb.com/jokes/random', params = {'limitTo': '[nerdy]'})
    # params = {'firstName': 'Johannes', 'lastName': 'Krohn'}
    # r = requests.get('http://api.icndb.com/jokes/random', params=params)

    r = requests.get('http://api.icndb.com/jokes/random', params={'limitTo':'[nerdy]'})
    r = r.json()
    joke = r['value']['joke']
    return joke

def traffic(api, message):
    """
    Act on the /traffic command. Post a few traffic cam images to a Cisco Spark space
    :param api: Spark API instance
    :param message: message object
    :return: markdown of text to be posted
    """

    # URLs of a few traffic cams in Germany
    german_traffic_cams = [
        'http://autobahn-rlp.de/syncdata/cam/380/thumb_640x480.jpg',
        'http://autobahn-rlp.de/syncdata/cam/385/thumb_640x480.jpg',
        'http://autobahn-rlp.de/syncdata/cam/165/thumb_640x480.jpg'
    ]
    room_id = message.roomId

    # need to post the attachments individually as the Cisco Spark API currently only supports one attachment at a time.
    for file in german_traffic_cams:
        api.messages.create(roomId=room_id, files=[file])

    return 'Traffic cam images posted above as requested'

def quote(message):
    r = requests.get('https://quotesondesign.com/wp-json/wp/v2/posts/?orderby=rand')
    r = r.json()
    r = random.choice(r)
    quote = r['content']['rendered']
    author = r['title']['rendered']
    r = f'{quote}\n\n{author}'
    return r


def number(api, message):
    """
    Get a fun fact for a number
    """
    m=re.match(r'.*/number(\s+\d+)?', message.text)
    try: 
        number = m.groups()[0]
        number = str(int(number))
    except (TypeError, ValueError, AttributeError):
        number = 'random'
        api.messages.create(roomId=message.roomId, 
                            text='No number provided. Getting fun fact for a randum number.')
    
    r = requests.get('http://numbersapi.com/{number}'.format(number=number))
    return r.text

def get_dilbert(api, message):
    m = re.match(r'.*/dilbert\s+(\S+)?', message.text)
    try:
        search_param = m.groups()[0]
    except (TypeError, ValueError, AttributeError):
        search_param = None

    if search_param is None:
        search_param = 'management'

    search_url = 'https://dilbert.com/search_results?terms={search_param}'.format(search_param=search_param)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, "html.parser")
    comics = soup.find_all('div', class_='comic-item-container')
    images = [urllib.parse.urljoin(search_url, c.attrs['data-image']) for c in comics]
    if not images:
        message = 'Sorry, couldn\'t find any Dilbert strip for your search term \'{search_param}\''.format(
            search_param=search_param)
    else:
        api.messages.create(roomId=message.roomId, files=[random.choice(images)])
        message = 'Here you go..'
    return message

def peanuts(message):
    """
    Get a random Peanuts comic from thge Peanuts web page and post that comic to the space
    """
    s = requests.Session()
    url = 'https://www.peanuts.com/comics/'
    r = s.get(url=url)
    soup = BeautifulSoup(r.text, "html.parser")
    comics = soup.find_all('span', class_='peanuts-comic-strip')
    images = [c.img for c in comics]

    # each image has a list of urls in the srcset attribute
    # <img width="855" height="588" src="https://www.peanuts.com/wp-content/uploads/2017/09/pe080629comb_hs-855x588.png" class="attachment-desktop-comic size-desktop-comic" alt="" srcset="https://www.peanuts.com/wp-content/uploads/2017/09/pe080629comb_hs-855x588.png 855w, https://www.peanuts.com/wp-content/uploads/2017/09/pe080629comb_hs-300x206.png 300w, https://www.peanuts.com/wp-content/uploads/2017/09/pe080629comb_hs-768x528.png 768w, https://www.peanuts.com/wp-content/uploads/2017/09/pe080629comb_hs-1024x704.png 1024w, https://www.peanuts.com/wp-content/uploads/2017/09/pe080629comb_hs-675x464.png 675w" sizes="(max-width: 855px) 100vw, 855px">
    srcsets = [i.get_attribute_list('srcset')[0] for i in images]

    # the urls are comma separated
    srcsets = [s.split(',') for s in srcsets]

    # each url entry is a space separated tuple of url and a width attribute
    # https://www.peanuts.com/wp-content/uploads/2017/09/pe080629comb_hs-1024x704.png 1024w
    srcsets = [list(map(lambda x: x.strip().split(' '), sl)) for sl in srcsets]
    srcsets = [{w: url for url, w in sl} for sl in srcsets]

    # we only want urls of 1024w images
    images = [sl.get('1024w') for sl in srcsets]
    images = [i for i in images if i is not None]

    if images:
        # we can't post the image using the reqular message.create call b/c the url obtained above only works if the
        # right cookie and a referer header is sent in the request. The Webex backend has no knowledge of this. Thus
        # the only way to make this work ist to get the image locally and then post the attachment unsnig a multi-part
        # mime message
        image = random.choice(images)
        headers = dict(referer='https://www.peanuts.com/comics/')
        r = s.get(image, headers=headers)

        # prepare the multipart body
        data = {
            'roomId': message.roomId,
            'text': 'Here you go',
            'files': ('Image.png', r.content, r.headers['content-type'])
        }
        multi_part = requests_toolbelt.MultipartEncoder(fields=data)

        headers = {'Content-Type': multi_part.content_type,
                   'Authorization': 'Bearer {}'.format(BOT_ACCESS_TOKEN)}

        r = requests.post('https://api.ciscospark.com/v1/messages', data=multi_part, headers=headers)
        message = 'How do you like that one?'
    else:
        message = 'Sorry, couldn\'t find any Peanuts comics'

    return message

ngrok = Ngrok(port=LOCAL_PORT)
ngrok_url = ngrok.start()
print('Public ngrok URL: {}'.format(ngrok_url))

# Create a new bot
bot = webexteamsbot.TeamsBot(BOT_APP_NAME, teams_bot_token=BOT_ACCESS_TOKEN,
                             teams_bot_url=ngrok_url, teams_bot_email=BOT_EMAIL, debug=True)

# Webex Teams API
api = webexteamssdk.WebexTeamsAPI(BOT_ACCESS_TOKEN)

# Add new command
bot.add_command('/chuck', 'get Chuck Norris joke', get_joke)
bot.add_command('/traffic', 'show traffic cams', functools.partial(traffic, api))
bot.add_command('/quote', 'get a random quote', quote)
bot.add_command('/number', 'get fun fact for a number', functools.partial(number, api))
bot.add_command('/dilbert', 'get random dilbert comic', functools.partial(get_dilbert, api))
bot.add_command('/peanuts', 'get random peanuts comic', peanuts)


# Run Bot
# Typically the bot would be run via:
# bot.run(host='0.0.0.0', port=LOCAL_PORT)
# .. but that causes a hickup when executed from a Jupyter environment.
# That's why we use below simple method to run the bot
logging.getLogger('werkzeug').setLevel(logging.ERROR)
werkzeug.serving.run_simple(hostname='0.0.0.0', port=LOCAL_PORT, application=bot)


Waiting for ngrok startup...
Trying to get tunnel information from ngrok client API
Trying to get tunnel information from ngrok client API
Public ngrok URL: https://6da11ebf.ngrok.io


Teams Bot Email: brkcol2175@webex.bot
Teams Token: REDACTED
Found existing webhook.  Updating it.
Configuring Webhook. 
Webhook ID: Y2lzY29zcGFyazovL3VzL1dFQkhPT0svZTQxNjc2MTQtZTc4OS00MTAzLTk5MWEtNzhkMGU0MGY4NGMx
Message content:
Webex Teams Message:
{
  "id": "Y2lzY29zcGFyazovL3VzL01FU1NBR0UvMzkzOTY2NTAtNDE5Zi0xMWVhLThmNDYtM2Q1NzRkNmM5ZDg4",
  "roomId": "Y2lzY29zcGFyazovL3VzL1JPT00vNjkyYzIzODAtNDE5ZS0xMWVhLTk0ZGEtNGIzMjE0YzIyMWVh",
  "roomType": "group",
  "text": "Testbot hi",
  "personId": "Y2lzY29zcGFyazovL3VzL1BFT1BMRS8wNzhkOGVjMi05Mjg5LTQ2NTUtOWE5NC0wNDNiOWVjMTMyOTk",
  "personEmail": "jkrohn@cisco.com",
  "html": "<p><spark-mention data-object-type=\"person\" data-object-id=\"Y2lzY29zcGFyazovL3VzL1BFT1BMRS9iNzczYWFmMC1lZDBiLTQ0MjctODk3ZC1mNmVlN2VmNmE0ZWE\">Testbot</spark-mention> hi</p>",
  "mentionedPeople": [
    "Y2lzY29zcGFyazovL3VzL1BFT1BMRS9iNzczYWFmMC1lZDBiLTQ0MjctODk3ZC1mNmVlN2VmNmE0ZWE"
  ],
  "created": "2020-01-28T07:24:30.389Z"
}
Message from: jkrohn@cisco.com
Messag